# ge_scraper

- Takes data from the offical runescape GE api and extracts price of an item at a certain timestamp for the past 180 days. 
- The timestamps are taken for each day at 20:00:00. The untis are in ms

In [5]:
import requests
import json
import csv

In [7]:
'''Convert ms time from API to local time'''
import datetime

# Given timestamp in milliseconds
timestamps = [1744502400000, 1744416000000]

for ts in timestamps:
    # Convert to seconds
    ts /= 1000
    # Convert to a readable datetime (local time)
    readable_time = datetime.datetime.fromtimestamp(ts)
    print(f"Local Time: {ts} -> {readable_time}")

Local Time: 1744502400.0 -> 2025-04-12 20:00:00
Local Time: 1744416000.0 -> 2025-04-11 20:00:00


In [ ]:
csvName = 'Rune_Data'

# get more item codes here: https://everythingrs.com/tools/osrs/itemlist/238
# itemList = [1521, 1519, 1517, 1515]  # Logs
itemList = list(range(554,567,1))  # Runes

fullDict = {}
labels = ['timestamp']


In [3]:
# Construct dictionary full of data
for itemID in itemList:
    r = requests.get(d'http://services.runescape.com/m=itemdb_oldschool/api/graph/{itemID}.json')
    json_data = json.loads(r.text)
    current_daily_dict = json_data['daily']

    for daily_timestamp in current_daily_dict:
        if (daily_timestamp in fullDict):
            fullDict[daily_timestamp].append(current_daily_dict[daily_timestamp])
        else:
            fullDict[daily_timestamp] = [current_daily_dict[daily_timestamp]]
    
    r2 = requests.get('http://services.runescape.com/m=itemdb_oldschool/api/catalogue/detail.json', params={'item': itemID})
    labels.append(json.loads(r2.text)['item']['name'].replace(" ", "_"))


# Write to CSV file
with open(f'data/ge/{csvName}.csv', mode='w', newline='') as GE_data:
    GE_writer = csv.writer(GE_data, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    GE_writer.writerow(labels)  # write field names

    for daily_timestamp in fullDict:
        new_array = [daily_timestamp]
        new_array.extend(fullDict[daily_timestamp])
        GE_writer.writerow(new_array)